# 1. Start with installing packages

First thing you need to do is ensure you have the right packages installed. Later you will call them into your code using the __using__ function. Here, we first call the Pkg package, then apply the add() fuction from that library to install other packages we will need as shown in the next block.

As we will be running a function from a Python library, we will also need to add the *scipy* package to the Julia's instance of Python. For that we will use the Conda package. 

To excecute a block, hit the __>| Run__ button from the above menu. If you already installed these packages before (e.g. directly from the Julia terminal), you don't need to run the following block because it will take ages to excecute!

In [ ]:
using Pkg # calls the Pkg package
Pkg.add("PyCall") # applies the add() function from the Pkg library to install PyCall. This will take a while!
Pkg.add("Glob")
Pkg.add("FileIO")
Pkg.add("PyPlot")
Pkg.add("Plots")
Pkg.add("DSP")
Pkg.add("Statistics")
Pkg.add("JLD2")

# you will also need to add the scipy package to python, for this we use Conda rather than Pkg:
using Conda 
Conda.add("scipy")

In [ ]:
pwd() # this tells you which directory you are in. 

# Make sure that your script and the data files are all in the same working directory

In [ ]:
varinfo()  # lets look at what we have in the workspace (variables in Julia's working memory)

# 2. Importing data

Here you will import data using Glob and FileIO packages. You can read up about these packages here if you wish: 

https://github.com/vtjnash/Glob.jl

https://github.com/JuliaIO/FileIO.jl 


In [ ]:
using Glob , FileIO
data_package=glob("neuro_data.jld2")
rawSIG = load(data_package[1], "rawSIG")
pretrial_time = load(data_package[1], "pretrial_time")
laser = hcat(load(data_package[1], "laser"))
trial_t1 = load(data_package[1], "trial_t")
fs = load(data_package[1], "fs")

vect_index=collect(1:size(rawSIG,1))  # generate a vector from 1:size of the rawSIG vector

In [ ]:
varinfo()  # lets look at what we have in the workspace (variables in Julia's working memory)

In [ ]:
size(rawSIG) 

# This will show you the size of the variable rawSIG, which should look like this: (401408,) 
# This means that the variable rawSIG is an array of size 401408 rows but no columns (i.e. it is a vector)

# 3. Visualising data with different plotting libraries available in Julia

Here we will look at <font color='blue'>PyPlot</font>  and <font color='blue'>Plots</font>. 

<font color='blue'>PyPlot</font> is a Python library which comes from matplotlib, so the syntax for <font color='blue'>PyPlot</font> in Julia is identical to the python version. <font color='blue'>PyPlot</font> was designed to be similar in use to MATLAB plots, and is therefore very popular. Read more here: https://github.com/JuliaPy/PyPlot.jl. A tutorial on the Python version can be found here: https://matplotlib.org/tutorials/introductory/pyplot.html where you can find extensive instructions. <font color='blue'>PyPlot</font> works by using <font color='blue'>PyCall</font>, a Julia package that runs an instance of Python inside Julia to run Python libraries. This is very convenient because Python is much more established than Julia, so <font color='blue'>PyCall</font> permits us to use anything from Python that isn't yet available in Julia. 

<font color='blue'>Plots</font> is a native Julia library which can have various backends, including Gr(), plotly(), and (confusingly), also a native Julia version of pyplot, which is called by pyplot(). You can read more here: https://docs.juliaplots.org/latest/

Below we will compare <font color='blue'>PyPlot</font> (probably the most versatile plotting library) with <font color='blue'>Plots</font>; plotly(), which is for generating plots in html. If we swap between different plotting backends, Julia freaks out, so you might need to restart Julia and start from the top!




In [ ]:
using PyPlot; pygui(true) # you need to add pygui(), otherwise the plots will be stored in memory but not shown.
# with pygui(), you will generate a new interactive window
figure() # this calls a new figure
PyPlot.plot(vect_index,rawSIG)
PyPlot.plot(vect_index,1000*laser)
PyPlot.legend(["rawSIG", "laser displacement"])

In [ ]:
using Plots; plotly() # here we are using the plotly backend. Try gr() or pyplot() instead
Plots.plot(vect_index,rawSIG, hover=vect_index, label = "rawSIG") #plot rawSIG (first row, all columns), display x variable on hover
Plots.plot!(vect_index,1000*laser, label = "laser displacement", hover=1000*laser[1,:])
# once plotted, you can interact with the plot using the tools above the plot which become visible as you hover your mouse over the plot

## 4. Filtering data

We will use the Digital signal processing package <font color='blue'>DSP</font> for filtering our data. More about what this package can do can be found here: https://github.com/JuliaDSP/DSP.jl. 



In [ ]:
using DSP  # this is calling the DSP package which has tools for digital signal processing
bpass = 300 # define Bandpass filter at 300 Hz
bstop = 5000 # define Bandstop filter at 5000 Hz
responsetype = Bandpass(bpass, bstop, fs = fs) # standard filtering use 300-5000
designmethod = Butterworth(6)
HFdata = filtfilt(digitalfilter(responsetype, designmethod), rawSIG)
negHFdata = -HFdata # convention is to invert signal of extracellular recordings (upward deflection represents action potentials)
varinfo() # view workspace

In [ ]:
# view signals:
figure()
PyPlot.plot(vect_index[209000:212000],rawSIG[209000:212000]) #plot rawSIG: first row, columns (samples) between 209000 and 212000
PyPlot.plot(vect_index[209000:212000],1000*laser[209000:212000])
PyPlot.plot(vect_index[209000:212000],negHFdata[209000:212000])
PyPlot.legend(["rawSIG",  "laser displacement", "negHFdata"] )

# 5. Reorganise our signal into multiple trials

In this experiment we evoked a mechanical pulse 10 times with an interval of about 1s between stimuli. 

Lets now lets chop up our signal based on the peak of each mechanical stimulus. For this, we need to determine where to split up each trial (how manny samples before and after the stimulus). Lets base the chopping point from the peak of our laser signal. 

To do this, we will a python function called *find_peaks* which comes from SciPy.org, freeware with lots of great tools written in Python, such as for signal processing: <font color='blue'>scipy.signal</font>. See: https://docs.scipy.org/doc/scipy/reference/tutorial/signal.html for more details on what is contained in <font color='blue'>scipy.signal</font>, and from that site you can find the **scipy.signal.find_peaks** page.

To access this Python library and function, we will use <font color='blue'>PyCall</font>, which will run a Python session inside Julia. If we wanted to, we could take the open source code and make our own *find_peak* fuction in Julia (someone has done this already: https://github.com/tungli/Findpeaks.jl), but here we are demonstrating how we can use <font color='blue'>PyCall</font> to run any Python code. 

In [ ]:
using PyCall # import the PyCall library, which allows us to use python commands

scisig = pyimport("scipy.signal") # import the scipy signal processing library from python. This contains the find_peaks function. 

stim_ind, pkHeights = scisig.find_peaks(1000*laser[:,1], height = 15, distance = 16000) 
# use the python find peak function to find all the indexes of peaks that are greater than 15 and only allow one peak per 16000 samples

In [ ]:
stim_ind

In [ ]:
pkHeights

# 6. Making a function

We will now make a fucntion that will chop up the signal according to the location of the found peaks. We will define this function as called *split_trials*. A function contain input argumnents and may or maynot have an output. In this case, we will take in some arguments that we need to generate the chopped sequence, so that we will convert a vector (single row) into an array of 10 vectors stacked on top of each other. 

If you provide a string before a function, you can access it from a help menu. So good coding practice will ensure our function well described: 



In [ ]:
"""
split_trials() takes a single vector and splits it into a 2D array according to the a vector
which contains the indexes of where to split the vector. You define how far before and after
the split location for each trial. This function also permits shifting relative to the split point.

%%%%%%% Inputs %%%%%%%

- amp_signal: is an array containing the data from the electrode of interest

- amp_samplerate: is a value containing the samplerate from recording

- stms: is a row vector that indicates the time that each stimulus artifact occurred in the recording

- trial_length: is the length of the trials that the data will be split into (in samples)

- Lag_time: how long after the stimulus artefact each trial will begin (usually '0', but can be
later for chopping up windows for machine learning etc)

- pre_trial_length: the amount of recording before the stimulus artefact to be included (in samples)

%%%%%%% Outputs %%%%%%%

- trial_data: an n*m matrix conatining all the data for each trial
     n is the number of trials and m is the length of the trials in sample
"""
function split_trials(amp_signal, amp_samplerate, stms, trial_length, lag_time, pre_trial_length)
    trial_data = [];
    # creating the matrices containg each trial as recorded by each of seven electrodes
    for k = 1:size(stms,2)
        strts = convert(Int,floor(stms[1,k]+lag_time)-pre_trial_length)
        t_end = convert(Int,strts+(trial_length-1))
        if t_end > length(amp_signal)
            println("The end of the final trial was too close to the end of the\n",
            "recording to acquire background data for the last signal") &&  break
        else
            # trial_data[k,:] = amp_signal[strts:t_end]
            push!(trial_data, amp_signal[strts:t_end])
        end
    end
    return trial_data
end


In [ ]:
# we can request docmentation for a fuction by using @doc macro. 
# You can provide documentation for your fuction by including it as a string enclosed by triple quotes as we did above. 

@doc split_trials

## Applying the new function

We will now apply our new function to split our signal (negHFdata) and the laser displacement analogue signal:

In [ ]:
trial_data = split_trials(negHFdata, fs, stim_ind', 5000,  0, 1000) # split the data around the laser peaks, make it a 5000 sample sequence
trial_laser = split_trials(laser, fs, stim_ind', 5000,  0, 1000) # split the laser signal around the laser peaks

# 7. View filtered signals

To plot any signls, we need x and y variables. To plot a signal against time, we have to generate a time vector, as follows: 

In [ ]:
time_vect=-1000:1:3999 # this is the 5000 vector starting from -1000 (as we had applied for our split_trial fuction)
time_vect = (time_vect/fs)*1000 # we need to convert each sample to the correct time stamp, so we need to know the sample frequency (fs)

Then we can plot the signals against time:

In [ ]:
#  view a single signal (individual axes)

#############  view signals (individual axes)
trial_to_inspect = 3 # here we can choose which trial we want to inspect, just change the value to between 1 and 10. 
figure()
PyPlot.plot(time_vect, trial_data[trial_to_inspect])
PyPlot.plot(time_vect, 1000*trial_laser[trial_to_inspect])

PyPlot.legend(["trial data", "displacement"])
PyPlot.xlabel("milliseconds")
PyPlot.ylabel("trial data (uV) | displacement (um)")
PyPlot.title("Signal from trial no. $trial_to_inspect") # putting a $ in front of a variable converts the value into a string

We can also view all 10 signals at once, using a <font color='blue'>**for**</font> loop:

In [ ]:
# view all trials a single figure with 10 subplots:

No_trials=size(trial_data,1) # number of trials
fig1, ax1 = subplots(No_trials, 1, sharex="all", sharey="all") # define subplots and which axes are shared (for zooming all at once)
for i = 1:No_trials # for each trail plot the data and the displacement
    ax1[i,1].plot(time_vect, trial_data[i])
    ax1[i,1].plot(time_vect, 1000*trial_laser[i])
    # ax1[i,1].plot((spike_ind[i]/30) .-300, pkH[i],"r.", markersize = 6, color = "red")
end

fig1.legend(["trial data", "displacement"])
fig1.text(0.5, 0.05, "milliseconds", ha="center", va="center")
fig1.text(0.05, 0.5, "trial data (uV) | displacement (um)", ha="center", va="center", rotation="vertical")
fig1.suptitle("Signals of $No_trials trials") # add title to subplots
fig1.legend(["trial data", "displacement (microns)"]) # add legend to subplot

# 8. Event detection

The easiest way to detect units is to select signals that reach a certain threshold. Here we will determine a threshold and detect signals that exceed this threshold. For that, we will calculate the threshold for some standard deviation above the mean of a signal over a range before the event has occured. Recall that our signals have 1000 samples before the stimulus, and 4000 after. So we can determine our background from the first ~1000 samples, as follows: 

In [ ]:
using Statistics
HFdata_mean=mean(trial_data) # this will generate a signal representing the mean of the 10 signals

In [ ]:
# now lets plot this for some examples: 

figure()
PyPlot.plot(time_vect,trial_data[1])
PyPlot.plot(time_vect,trial_data[5])
PyPlot.plot(time_vect,trial_data[10])
PyPlot.plot(time_vect,HFdata_mean, color="black", linewidth=2)
PyPlot.legend(["trial 1", "trial 5", "trial 10", "mean of 10 trials"])
# we can see this doesnt capture all the signals well because of the jitter (need to zoom in)

Lets now calculate the mean over the background range (i.e. first 1000 samples), and define a threshold to be 3 standard deviations greater than the mean:

In [ ]:
bkgnd_1= 3 * std(trial_data[1][1:1000])

Now lets plot the backgorund threshold and the peaks above this threshold on the same plot (if you closed the previous plot, generate the figure again before you execute the next code block).

In [ ]:
# now lets find all peaks greater than bkgnd, and plot on same graph as before (note we arn't calling a new figure):

ind_x, pks_x = scisig.find_peaks(trial_data[1], height = bkgnd_1) # use "find_peaks" function from python scisig library to find all peaks in x greater than the height bkgnd_1
PyPlot.plot(time_vect,bkgnd_1*ones(5000,1), label = "3x SD above background noise (bkgnd_1)")
PyPlot.plot((ind_x .-1000)/fs*1000,pks_x["peak_heights"], "r.", markersize = 6, color = "red")
PyPlot.legend(["trial 1", "trial 5", "trial 10", "mean of 10 trials","detection threshold", "peaks above threshold"])
# this last expression gets the index for the peak, subtracts 1000 because we are converting to time and the vector starts at -1000 samples from time zero
# we then divide by the sample frequency (fs) x1000 to convert our x-axis to ms rather than indicies of the vector.


In [ ]:
#lets do this for all trials

inds_trials = [] # need to declare the variable outside a for loop becauase we will be calling in from inside the for loop
pks_trials = [] 
for i = 1: 10
    ind_x, pks_x = scisig.find_peaks(trial_data[i], height = 3 * std(trial_data[i][1:1000]))
    push!(inds_trials, ind_x')
    push!(pks_trials, pks_x["peak_heights"]')
end

In [ ]:
inds_trials

In [ ]:
pks_trials

In [ ]:
# test it works on a random example:

figure()
PyPlot.plot(time_vect,trial_data[7])
PyPlot.plot((inds_trials[7] .-1000)[1,:]/fs*1000,pks_trials[7], "r.", markersize = 6, color = "red")
# looks ok; try substituting another number instead of 7...
# can anyone tell me why we need the [1,:]?

In [ ]:
# lets plot all the signals on separate subplots:

fig2, ax2 = subplots(No_trials, 1, sharex="all", sharey="all") # define subplots and which axes are shared
for i = 1:No_trials # for each trail plot the data and the displacement
    ax2[i,1].plot(time_vect, trial_data[i])
    ax2[i,1].plot(time_vect, 1000*trial_laser[i])
    ax2[i,1].plot((inds_trials[i] .-1000)[1,:]/fs*1000, pks_trials[i],"r.", markersize = 6, color = "red")
end
fig2.suptitle("Signals of $No_trials trials") # add title to subplots
fig2.legend(["trial data", "displacement (microns)", "detected peaks"]) # add legend to subplot

# 9. Building and quantifying raster plots

To perform some analysis on the neuronal behavoural response to the stimulus, it is convenient to observe the pattern of firing relative to multiple trials. A rasterplot is a great way to observe this, and a frequency histogram is a great way to quantify the activty from multiple trials. 

In [ ]:
# Lets make a raster for all 10 trials and see what it looks like

figure()
for i =1:No_trials
PyPlot.plot((inds_trials[i] .-1000)/fs*1000, i*ones(size(pks_trials[i])),"r.", markersize = 6, color = "red")
end
# note that the raster is in the reverse order to the previous figure; why?

In [ ]:
# Lets add a frequency histogram to count the number of events above threshold:

binwidth=1 #define bin width, you can change this and see how it looks
pretrial_length = 1000 # each trial has 1000 samples before it
trial_len = 5000 # each trial has 5000 samples in total
bin_edges=collect(round(-pretrial_length/fs*1000):binwidth: round((-pretrial_length+trial_len)/fs*1000))

inds_trials_flat = sort(reduce(hcat,inds_trials)') # concatinate all vectors horixontally, then sort in order
inds_trials_flat = inds_trials_flat .-1000 # subtract 1000 (range = -1000:4000, i.e. 5000 length vector)

hist(inds_trials_flat/fs*1000, bins=bin_edges, alpha = 0.5) # generate a histogram and associated data. 

In [ ]:
## now add to the raster and link the axes
fig3, ax3 = subplots(2, 1, sharex="all") # start a new plot
for i =1:No_trials
ax3[1,1].plot((inds_trials[i] .-1000)/fs*1000, i*ones(size(pks_trials[i])),"r.", markersize = 6, color = "red")
end
ax3[2,1].hist(inds_trials_flat/fs*1000, bins=bin_edges, alpha = 0.5)


ax3[1,1].set(ylabel = "Trial number")
ax3[2,1].set(ylabel = "Frequency")
ax3[1,1].legend(["rasterplot of 10 trials"]) 
ax3[2,1].legend(["histogram of rasterplot"]) 

fig3.text(0.5, 0.05, "milliseconds", ha="center", va="center")
fig3.text(-50, 0.5, "Trial number", ha="center", va="center")
fig3.suptitle("Signals of $No_trials trials") 